# Fake Job Postings

Ссылка на соревнование: [Fake Job Postings](https://www.kaggle.com/c/rucode-fake-job-postings/overview/evaluation)

В этом конкурсе необходимо решить задачу выявления фейковых объявлений о приеме на работу. Данные представляют собой csv таблицу с текстовыми и категориальными полями. Чтобы успешно решить эту задачу, понадобится обработать оба вида информации.

# RoBERTa-large

Для решения задачи я использовала предобученную модель **RoBERTa** из библиотеки **Hugging Face**.

Подробнее о модели можно прочитать по ссылкам:


*   [RoBERTa: A Robustly Optimized BERT Pretraining Approach](https://arxiv.org/abs/1907.11692)

*   [RoBERTa-large](https://huggingface.co/roberta-large)



## Dependencies

Установим библиотеки от Hugging Face

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install -U nlp

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-va80aw1c
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-va80aw1c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 3.4MB/s 
     |████████████████████████████████| 3.0MB 10.6MB/s 
     |████████████████████████████████| 1.1MB 34.6MB/s 
  Created wheel for transformers: filename=transformers-3.3.0-cp36-none-any.whl size=1059443 sha256=bc1939c66302fc5055064f7fc3781fe9e6a111aaef8193aa5e7fcecf641e0d74
  Stored in directory: /tmp/pip-ephem-wheel-cache-66rdrse5/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=cf444d67bf9e6058b1ef3176e0d915bc266797c82cc415f1a67209fdd48f53aa
  Stored in directory: /root/.cache

Для работы на TPU установим PyTorch-XLA

In [ ]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version 20200924  --apt-packages libomp5 libopenblas-dev

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5115  100  5115    0     0  24127      0 --:--:-- --:--:-- --:--:-- 24127
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200924 ...
     |████████████████████████████████| 61kB 3.0MB/s 
Uninstalling torch-1.6.0+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
Done updating TPU runtime
  Successfully uninstalled torch-1.6.0+cu101
Uninstalling torchvision-0.7.0+cu101:
  Successfully uninstalled torchvision-0.7.0+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly+20200924-cp36-cp36m-linux_x86_64.whl...
\ [1 files][113.8 MiB/113.8 MiB]                                                
Operation completed over 1 objects/113.8 MiB.                                    


Импортируем необходимые библиотеки

In [ ]:
import os
os.environ['XLA_USE_BF16'] = "1"
os.environ['XLA_TENSOR_ALLOCATOR_MAXSIZE'] = '100000000'

import gc
gc.enable()

import numpy as np
import pandas as pd

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader #, Dataset
from torch.utils.data.distributed import DistributedSampler

import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.utils.serialization as xser
import torch_xla.version as xv

import warnings
warnings.filterwarnings("ignore")

Зафиксируем `seed` для воспроизводимости результата

In [ ]:
import random

def my_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

my_seed()

## Model

Загружаем предобученную модель `roberta-large` и токенизатор

In [ ]:
WRAPPED_MODEL = xmp.MpModelWrapper(AutoModelForSequenceClassification.from_pretrained("roberta-large"))
tokenizer = AutoTokenizer.from_pretrained("roberta-large")

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

## Data preprocessing

### Train & Val

Считываем данные для обучения

In [ ]:
train_df = pd.read_csv('train_data.csv')

Колонки:
*  **Id** - уникальный идентификатор объявления
*  **Название** - название вакансии
*  **Место** - место работы (город, страна, штат)
*  **Отдел** - название отдела в компании, куда есть вакансия
*  **Зарплата** - предлагаемая зарплата на вакансию
*  **Описание компании** - описание компании, которая разместила вакансию. Составлено самой компанией
*  **Описание вакансии** - развернутое описание вакансии. Составлено компанией, которая разместила вакансию.
*  **Требования** - описание обязанностей работника
*  **Соцпакет** - все benefits, предоставляемые компанией работнику
*  **Дистанционно** - бинарная переменная: 1, если работа удаленная
*  **Вопросы** - бинарная переменная: 1, если в первичной анкете при приеме на работу присутствуют вопросы
*  **Тип занятости** - полный/неполный рабочий день, стажировка и т.д.
*  **Опыт** - опыт, которым должен обладать потенциальный кандидат
*  **Образование** - образование, которым должен обладать потенциальный кандидат
*  **Индустрия** - сфера деятельности компании. Ритейл/медицина и т.д.
*  **Позиция** - позиция, на которую открыта вакансия
*  **Фейк** - (только в тренировочном датасете) целевая переменная: 0, если объявление не фейк, 1, если фейк

In [ ]:
train_df.head()

,Id,Название,Место,Отдел,Зарплата,Описание компании,Описание вакансии,Требования,Соцпакет,Дистанционно,Вопросы,Тип занятости,Опыт,Образование,Индустрия,Позиция,Фейк
0,10497,"Reporter, Agri Investor","US, ,",KHR,NaN,PEI Media (#URL_4cffe430b8da4c0d7e6074d11e638d...,Seeking savvy business reporter Are you a news...,The successful candidate should have a bachelo...,PEI Media (#URL_4cffe430b8da4c0d7e6074d11e638d...,0,0,Full-time,Associate,Bachelor's Degree,Publishing,Writing/Editing,0
1,8219,Software Engineer,"US, VA, Herndon",Software Engineering,NaN,Roka Security is a boutique security firm that...,Roka Security is looking for Software Engineer...,NaN,NaN,0,0,Full-time,Not Applicable,Unspecified,Computer Software,Engineering,0
2,5342,Structural Engineer,"NL, ZH, Rotterdam",onshore,NaN,NaN,Job tasks:Analyse project specifications on st...,Job requirements:HTS/TU Civil Engineering or M...,NaN,0,1,NaN,NaN,NaN,NaN,NaN,0
3,1916,Senior Publishing Specialist,"US, ,",NaN,NaN,NaN,The Senior Publishing Specialist will be part ...,Minimum of 3 – 4 years experience in Regulator...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0
4,1811,Sales Representative,"CA, BC, Vancouver",Sales & Marketing,NaN,NaN,BCCC Enterprises provides management consultin...,Key Responsibilities:Contribute to the financi...,How to ApplyThis is an exciting opportunity. ...,0,1,NaN,NaN,NaN,NaN,NaN,0


Видим, что данные содержат достаточно много пустых значений

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10728 entries, 0 to 10727
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Id                 10728 non-null  int64 
 1   Название           10728 non-null  object
 2   Место              10504 non-null  object
 3   Отдел              3758 non-null   object
 4   Зарплата           1730 non-null   object
 5   Описание компании  8682 non-null   object
 6   Описание вакансии  10728 non-null  object
 7   Требования         9097 non-null   object
 8   Соцпакет           6397 non-null   object
 9   Дистанционно       10728 non-null  int64 
 10  Вопросы            10728 non-null  int64 
 11  Тип занятости      8586 non-null   object
 12  Опыт               6478 non-null   object
 13  Образование        5836 non-null   object
 14  Индустрия          7775 non-null   object
 15  Позиция            6853 non-null   object
 16  Фейк               10728 non-null  int64

Данные сильно несбалансированы: среди объявлений фейковых очень мало (примерно 5%)

In [ ]:
train_df['Фейк'].value_counts()

0    10205
1      523
Name: Фейк, dtype: int64

Выделяются две колонки **Дистанционно** и **Вопросы**, в которых данные числовые. Представим их в строковом виде, заменив на подходящие по смыслу фразы, чтобы можно было подавать их на вход нашей модели наравне с остальными колонками.

In [ ]:
train_df['Дистанционно'] = train_df['Дистанционно'].map({1:'Telecommuting positions', 0:'Office positions'})

In [ ]:
train_df['Дистанционно'].value_counts()

Office positions           10274
Telecommuting positions      454
Name: Дистанционно, dtype: int64

In [ ]:
train_df['Вопросы'] = train_df['Вопросы'].map({1:'Screening questions are present', 0:'There are no screening questions'})

In [ ]:
train_df['Вопросы'].value_counts()

There are no screening questions    5424
Screening questions are present     5304
Name: Вопросы, dtype: int64

Выделим колонки, которые хотим подать на вход нашей модели

In [ ]:
columns_to_text = ['Название', 
                   'Тип занятости',
                   'Дистанционно',
                   'Вопросы', 
                   'Описание компании', 
                   'Описание вакансии', 
                   'Требования']

Также соберем все колонки, кроме целевой переменной

In [ ]:
old_columns = train_df.columns

In [ ]:
old_columns = old_columns.drop('Фейк')

In [ ]:
old_columns

Index(['Id', 'Название', 'Место', 'Отдел', 'Зарплата', 'Описание компании',
       'Описание вакансии', 'Требования', 'Соцпакет', 'Дистанционно',
       'Вопросы', 'Тип занятости', 'Опыт', 'Образование', 'Индустрия',
       'Позиция'],
      dtype='object')

Все пропуски в данных заменим на пустую строку

In [ ]:
train_df.fillna('', inplace=True)

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10728 entries, 0 to 10727
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Id                 10728 non-null  int64 
 1   Название           10728 non-null  object
 2   Место              10728 non-null  object
 3   Отдел              10728 non-null  object
 4   Зарплата           10728 non-null  object
 5   Описание компании  10728 non-null  object
 6   Описание вакансии  10728 non-null  object
 7   Требования         10728 non-null  object
 8   Соцпакет           10728 non-null  object
 9   Дистанционно       10728 non-null  object
 10  Вопросы            10728 non-null  object
 11  Тип занятости      10728 non-null  object
 12  Опыт               10728 non-null  object
 13  Образование        10728 non-null  object
 14  Индустрия          10728 non-null  object
 15  Позиция            10728 non-null  object
 16  Фейк               10728 non-null  int64

Теперь объединим значения из выбранных колонок **columns_to_text** в одну строку и запишем ее в новую колонку **text_data**

In [ ]:
train_df['text_data'] = train_df[columns_to_text].agg(' '.join, axis=1)

In [ ]:
from pprint import pprint

Вот что у нас получилось. Не очень красиво, но "чистка" текста качество классификации только ухудшила (по крайней мере, у меня)

In [ ]:
for i in range(10):
    pprint(train_df['text_data'][i])
    print()

('Reporter, Agri Investor Full-time Office positions There are no screening '
 'questions PEI Media\xa0'
 '(#URL_4cffe430b8da4c0d7e6074d11e638d28a1e165c8782b891e111fc17606c3d961#) '
 'provides a range of highly respected publications and market leading events '
 'to its financial sector audience. The\xa0leading information provider in '
 'private capital markets, PEI Media also has a strong international presence, '
 'with offices in London, New York and Hong Kong. Seeking savvy business '
 'reporter Are you a news-hungry journalist? Have a passion for finance, a '
 'talent for breaking news, networking and developing sophisticated sources?If '
 'so, there’s an exciting full-time reporter position available with PEI '
 'Media, the leading global B2B financial publishing and information group '
 'focused exclusively on alternative asset classes.Based in Manhattan, this '
 'opening is a rare opportunity for an ambitious individual seeking to advance '
 'their career in one of the most dy

Теперь можем разделить данные, выделив для валидации 1024 примера

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_df, val_df = train_test_split(train_df, test_size=1024, random_state=42)

In [ ]:
val_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1024 entries, 592 to 6311
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Id                 1024 non-null   int64 
 1   Название           1024 non-null   object
 2   Место              1024 non-null   object
 3   Отдел              1024 non-null   object
 4   Зарплата           1024 non-null   object
 5   Описание компании  1024 non-null   object
 6   Описание вакансии  1024 non-null   object
 7   Требования         1024 non-null   object
 8   Соцпакет           1024 non-null   object
 9   Дистанционно       1024 non-null   object
 10  Вопросы            1024 non-null   object
 11  Тип занятости      1024 non-null   object
 12  Опыт               1024 non-null   object
 13  Образование        1024 non-null   object
 14  Индустрия          1024 non-null   object
 15  Позиция            1024 non-null   object
 16  Фейк               1024 non-null   int64

Удалим все старые колонки и наши валидационные данные готовы!

In [ ]:
val_df.drop(columns=old_columns, inplace=True)

In [ ]:
val_df

,Фейк,text_data
592,0,Technical Site Representative Full-time Offic...
608,0,Communications / Media Relations Full-time Of...
424,0,Customer Support Associate Office positions S...
10133,0,Property Manager (LCAM) Full-time Office posit...
5635,0,Talent Acquisition Specialist USA Full-time Of...
...,...,...
5269,0,Marketing Manager Full-time Office positions T...
7877,0,Healthcare Assistant Office positions There a...
4076,0,Clinical Psychotherapist Full-time Office posi...
106,0,Accountant / Office Manager Full-time Office p...


А вот в тренировочных надо еще устранить дисбаланс. 

Выделим отдельно только фейковые объявления

In [ ]:
fake_df = train_df[train_df['Фейк']==1]

In [ ]:
fake_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 466 entries, 9413 to 2433
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Id                 466 non-null    int64 
 1   Название           466 non-null    object
 2   Место              466 non-null    object
 3   Отдел              466 non-null    object
 4   Зарплата           466 non-null    object
 5   Описание компании  466 non-null    object
 6   Описание вакансии  466 non-null    object
 7   Требования         466 non-null    object
 8   Соцпакет           466 non-null    object
 9   Дистанционно       466 non-null    object
 10  Вопросы            466 non-null    object
 11  Тип занятости      466 non-null    object
 12  Опыт               466 non-null    object
 13  Образование        466 non-null    object
 14  Индустрия          466 non-null    object
 15  Позиция            466 non-null    object
 16  Фейк               466 non-null    int64

И добавим фейковые данные в наш тренировочный датасет столько раз, чтобы соотношение между настоящими и фейковыми стало примерно одинаковым

In [ ]:
train_df = pd.concat([train_df] + [fake_df]*18, axis=0, ignore_index=True)

In [ ]:
train_df.shape

(18092, 18)

Удалим все старые колонки (согласна, можно было сделать это и раньше)

In [ ]:
train_df.drop(columns=old_columns, inplace=True)

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18092 entries, 0 to 18091
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Фейк       18092 non-null  int64 
 1   text_data  18092 non-null  object
dtypes: int64(1), object(1)
memory usage: 282.8+ KB


Так намного лучше

In [ ]:
train_df['Фейк'].value_counts()

0    9238
1    8854
Name: Фейк, dtype: int64

### Test

Повторяем те же действия с тестовым датасетом.

Прочитаем данные

In [ ]:
test_df = pd.read_csv('test_data.csv')

In [ ]:
test_df.head()

,Id,Название,Место,Отдел,Зарплата,Описание компании,Описание вакансии,Требования,Соцпакет,Дистанционно,Вопросы,Тип занятости,Опыт,Образование,Индустрия,Позиция
0,10728,Internet Marketing Specialist,"IE, L, Dublin",Marketing,NaN,NaN,This role represents an important position wit..., 3 to 10+ years experience in a similar inter...,RemunerationA highly attractive salary will be...,0,0,NaN,NaN,NaN,NaN,NaN
1,10729,Business Development,"DE, , Berlin",NaN,NaN,We are on a mission: Making cities smarter.Cit...,You have an opinion about how we can make our ...,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
2,10730,Sonographer - Full and Part Time Opportunities,"GB, ,",Ultrasound,NaN,Diagnostic Health deploy and manage high quali...,We are a leading local ultrasound company spec...,Diploma of College of Radiographers or BSc (Ho...,Competative SalaryHours to suit,0,1,Part-time,NaN,Associate Degree,Medical Practice,Health Care Provider
3,10731,Care Support Worker,"GB, EDH, Edinburgh",NaN,NaN,Social Care Alba is the highest graded Care at...,Social Care Alba is the highest quality Care a...,Key Accountabilities would be:To provide indiv...,This is your chance to become part of our cont...,0,1,NaN,Not Applicable,NaN,Hospital & Health Care,NaN
4,10732,Outside Sales Professional-Springfield,"US, IL, Springfield",NaN,NaN,"ABC Supply Co., Inc. is the nation’s largest w...","As an Outside Sales Representative, you must h...",Track Record of Sales Success – B2B or B2CNo m...,"As an Outside Sales Representative, you will r...",0,0,NaN,NaN,NaN,NaN,NaN


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7152 entries, 0 to 7151
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Id                 7152 non-null   int64 
 1   Название           7152 non-null   object
 2   Место              7030 non-null   object
 3   Отдел              2575 non-null   object
 4   Зарплата           1138 non-null   object
 5   Описание компании  5890 non-null   object
 6   Описание вакансии  7151 non-null   object
 7   Требования         6088 non-null   object
 8   Соцпакет           4273 non-null   object
 9   Дистанционно       7152 non-null   int64 
 10  Вопросы            7152 non-null   int64 
 11  Тип занятости      5823 non-null   object
 12  Опыт               4352 non-null   object
 13  Образование        3939 non-null   object
 14  Индустрия          5202 non-null   object
 15  Позиция            4572 non-null   object
dtypes: int64(3), object(13)
memory usage: 894.

Преобразуем значения в колонках **Дистанционно** и **Вопросы**

In [ ]:
test_df['Дистанционно'] = test_df['Дистанционно'].map({1:'Telecommuting positions', 0:'Office positions'})

In [ ]:
test_df['Вопросы'] = test_df['Вопросы'].map({1:'Screening questions are present', 0:'There are no screening questions'})

Заменим пропуски

In [ ]:
test_df.fillna('', inplace=True)

Добавим новую колонку с объединенным текстом из выбранных колонок

In [ ]:
test_df['text_data'] = test_df[columns_to_text].agg(' '.join, axis=1)

Удалим все старые колонки

In [ ]:
test_df.drop(columns=old_columns, inplace=True)

In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7152 entries, 0 to 7151
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text_data  7152 non-null   object
dtypes: object(1)
memory usage: 56.0+ KB


## Datasets

С помощью токенизатора преобразуем наши тексты в вектора `input_ids` одной длины (512) и переведем их в необходимый для нашей модели формат тензоров `torch`

In [ ]:
import nlp

In [ ]:
MAX_LEN = 512 

def convert_to_features(batch):

    encodings = tokenizer.batch_encode_plus(batch['text_data'], 
                                            add_special_tokens=True,
                                            pad_to_max_length=True,
                                            truncation=True,
                                            max_length=MAX_LEN)
    return encodings

train_dataset = nlp.Dataset.from_pandas(train_df) 
train_dataset = train_dataset.map(convert_to_features, batched=True)
train_dataset.set_format("torch", columns=['input_ids', 'attention_mask', 'Фейк'])

valid_dataset = nlp.Dataset.from_pandas(val_df)
valid_dataset = valid_dataset.map(convert_to_features, batched=True)  
valid_dataset.set_format("torch", columns=['input_ids', 'attention_mask', 'Фейк'])

test_dataset = nlp.Dataset.from_pandas(test_df) 
test_dataset = test_dataset.map(convert_to_features, batched=True)  
test_dataset.set_format("torch", columns=['input_ids', 'attention_mask'])

## Train & Submit

### Metrics

Во время обучения будем следить за значениями функции потерь и метрикой f1

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
class AverageMeter(object):
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {avg' + self.fmt + '}'
        return fmtstr.format(**self.__dict__)

class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

def F1_score(outputs, targets):
    with torch.no_grad():
        return f1_score(targets, outputs.argmax(dim=-1))  

### Training

Опишем функцию для обучения

In [ ]:
def train_loop_fn(train_loader, model, optimizer, device, epoch=None):

    model.train()
    
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter(' F1', ':.2f')
    progress = ProgressMeter(
        len(train_loader),
        [losses, top1],
        prefix="[xla:{}]  Train:  Epoch: [{}]".format(xm.get_ordinal(), epoch + 1)
    )    

    for i, data in enumerate(train_loader):

        input_ids = data["input_ids"].to(device, dtype=torch.long)
        attention_mask = data["attention_mask"].to(device, dtype=torch.long)
        labels = data["Фейк"].to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(
            input_ids = input_ids,
            attention_mask = attention_mask,
            labels = labels
        )

        loss = outputs[0] 
        loss.backward()
        xm.optimizer_step(optimizer)

        f1 = F1_score(outputs[1].cpu(), labels.cpu())
        losses.update(loss.item(), input_ids.size(0))
        top1.update(f1.item(), input_ids.size(0))

        if i == len(train_loader) - 1: 
            progress.display(i)

    del loss
    del f1
    del outputs
    del input_ids
    del attention_mask
    del labels
    gc.collect()

### Evaluation

Опишем функцию для валидации

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
def eval_loop_fn(test_data_loader, model, device):

    model.eval()

    with torch.no_grad():

        test_preds = []
        targets = []
        val_loss = 0

        for i, data in enumerate(test_data_loader):

            input_ids = data["input_ids"].to(device, dtype=torch.long)
            attention_mask = data["attention_mask"].to(device, dtype=torch.long)
            labels = data["Фейк"].to(device, dtype=torch.float)

            outputs = model(
                input_ids = input_ids,
                attention_mask = attention_mask,
                labels = labels
            )

            val_loss += outputs[0].item()
            test_preds.extend(torch.argmax(outputs[1], -1).cpu().detach().tolist())
            targets.extend(data["Фейк"].detach().tolist())  
        
    xm.master_print(f'Val loss: {val_loss / len(test_data_loader)}')
    xm.master_print(f'F1 score: {f1_score(targets, test_preds)}')
    xm.master_print('Confusion matrix:')
    xm.master_print(f'{confusion_matrix(targets, test_preds)}')

    del outputs
    del targets
    del val_loss
    del test_preds
    del input_ids
    del attention_mask
    del labels
    gc.collect()

### Test

Опишем функцию, которая будет получать предсказания для тестового датасета и записывать их в колонку `Фейк` файла `sample_submission.csv` под именем `submission_epoch.csv`

In [ ]:
sample_submission = pd.read_csv('sample_submission.csv')

In [ ]:
def test_loop_fn(test_data_loader, model, device, epoch=0):

    model.eval()

    with torch.no_grad():

        test_preds = []

        for i, data in enumerate(test_data_loader):

            input_ids = data["input_ids"].to(device, dtype=torch.long)
            attention_mask = data["attention_mask"].to(device, dtype=torch.long)

            outputs = model(
                input_ids = input_ids,
                attention_mask = attention_mask,
            )[0]
            
            test_preds.extend(torch.argmax(outputs, -1).cpu().detach().tolist())  

    sample_submission['Фейк'] = test_preds
    sample_submission.to_csv(f'submission_{epoch + 1}.csv', index=False)

    del outputs
    del test_preds
    del input_ids
    del attention_mask
    gc.collect()

### Run

Укажем параметры обучения

In [ ]:
my_seed(42)

TRAIN_BATCH_SIZE = 4 * xm.xrt_world_size()
VALID_BATCH_SIZE = 8 
TEST_BATCH_SIZE = 8
EPOCHS = 10
LR = 5e-5 * xm.xrt_world_size() 

Определим загрузчики данных для каждого датасета и опишем основной цикл обучения. После каждой эпохи будем оценивать качество на валидационном датасете и делать предсказания для тестового.

In [ ]:
def _run():

    # Train
    xm.master_print('Starting Run ...')
    train_sampler = DistributedSampler( 
        train_dataset, 
        num_replicas=xm.xrt_world_size(),
        rank=xm.get_ordinal(),
        shuffle=True 
    )    
    train_data_loader = DataLoader(
        train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        sampler=train_sampler,
        drop_last=True,
        num_workers=0,
    )
    xm.master_print('Train Loader Created.')
    
    # Valid    
    valid_data_loader = DataLoader(
        valid_dataset,
        batch_size=VALID_BATCH_SIZE,
        drop_last=False,
        num_workers=0,
        shuffle=False,
    )
    xm.master_print('Valid Loader Created.')

    # Test
    test_data_loader = DataLoader(
        test_dataset, 
        batch_size=TEST_BATCH_SIZE,
        drop_last=False,
        num_workers=0,
        shuffle=False,
    )
    xm.master_print('Test Loader Created.')
    
    num_train_steps = int(train_dataset.num_rows / TRAIN_BATCH_SIZE / xm.xrt_world_size())
    device = xm.xla_device()
    model = WRAPPED_MODEL.to(device)
    xm.master_print('Done Model Loading.')
    optimizer = AdamW(
        model.parameters(),
        lr=LR, 
    )
    xm.master_print(f'Num Train Steps= {num_train_steps}, XRT World Size= {xm.xrt_world_size()}.')
    
    for epoch in range(EPOCHS):

        train_sampler.set_epoch(epoch)

        para_loader = pl.ParallelLoader(train_data_loader, [device])
        xm.master_print('Parallel Loader Created. Training ...')
        train_loop_fn(para_loader.per_device_loader(device),
                      model,  
                      optimizer, 
                      device,  
                      epoch
                     )        
        xm.master_print("Finished training epoch {}".format(epoch + 1))
            
        para_loader = pl.ParallelLoader(valid_data_loader, [device])
        xm.master_print('Parallel Loader Created. Validating ...')
        eval_loop_fn(para_loader.per_device_loader(device), 
                     model,  
                     device
                    )
        
        para_loader = pl.ParallelLoader(test_data_loader, [device])
        xm.master_print('Parallel Loader Created. Testing ...')
        test_loop_fn(para_loader.per_device_loader(device), 
                    model,  
                    device,
                    epoch 
                    )
        xm.master_print('Predicted.')

        # При необходимости можно сохранить веса модели
        # if epoch == EPOCHS-1:
        #     xm.save(model.state_dict(), f'RoBERTa_{epoch + 1}.pt')
        #     xm.master_print('Model Saved.')

Запускаем обучение

In [ ]:
def _mp_fn(rank, flags):
    _run()

FLAGS={}
xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method='fork')

Starting Run ...
Train Loader Created.
Valid Loader Created.
Test Loader Created.
Done Model Loading.
Num Train Steps= 565, XRT World Size= 8.
Parallel Loader Created. Training ...
[xla:0]  Train:  Epoch: [1][564/565]	Loss 1.2175e-01	 F1 0.87
[xla:4]  Train:  Epoch: [1][564/565]	Loss 1.1992e-01	 F1 0.89
[xla:2]  Train:  Epoch: [1][564/565]	Loss 1.1932e-01	 F1 0.88
[xla:3]  Train:  Epoch: [1][564/565]	Loss 1.2601e-01	 F1 0.87
[xla:7]  Train:  Epoch: [1][564/565]	Loss 1.2366e-01	 F1 0.87
[xla:1]  Train:  Epoch: [1][564/565]	Loss 1.1027e-01	 F1 0.87
[xla:6]  Train:  Epoch: [1][564/565]	Loss 1.2880e-01	 F1 0.86
[xla:5]  Train:  Epoch: [1][564/565]	Loss 1.1696e-01	 F1 0.86
Finished training epoch 1
Parallel Loader Created. Validating ...
Val loss: 0.043856799602508545
F1 score: 0.8807339449541285
Confusion matrix:
[[963   4]
 [  9  48]]
Parallel Loader Created. Testing ...
Predicted.
Parallel Loader Created. Training ...
[xla:2]  Train:  Epoch: [2][564/565]	Loss 1.6299e-02	 F1 0.93
[xla:3] 

Самые высокие результаты получились после 7, 8 и 9 эпох (соответственно, `submission_7.csv`, `submission_8.csv` и `submission_9.csv`)